In [ ]:
# you may need to `pip install --upgrade transformers`
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM
import torch


In [ ]:
# this block will downlaod the model from huggingface.co and cache locally
# run this block once to download the model

# to download different models, change the model_name after AI-Sweden-Models/{MODEL_NAME}
# you can find the other models here: https://huggingface.co/AI-Sweden-Models/gpt-sw3-126m
model_name = 'AI-Sweden-Models/gpt-sw3-126m-instruct'

# download the model and tokenizer from huggingface.co and cache locally
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# save the model and tokenizer locally
MODEL_PATH = "gptsw3model126m"
model.save_pretrained(MODEL_PATH)
tokenizer.save_pretrained(MODEL_PATH)


In [ ]:
# load the model and tokenizer from local cache

model = AutoModelForCausalLM.from_pretrained(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

model.eval()

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)


In [ ]:

promptInput = "Träd är fina för att"

# this will be different for each model
prompt = f"""
<|endoftext|><s>
User:
{promptInput}

<s>
Bot:
""".strip()

input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(device)

generated_token_ids = model.generate(
    inputs=input_ids,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.6,
    top_p=1,
)[0]

generated_text = tokenizer.decode(generated_token_ids)    

# code to extract text after "Bot:"
bot_text_start = generated_text.find("Bot:")
if bot_text_start != -1:
    bot_text = generated_text[bot_text_start + len("Bot:"):].strip()
    # stop after <s>
    bot_text = bot_text.split("<s>")[0].strip()
    print(bot_text)
else:
    print("Bot response not found in the generated text.")
